In [ ]:
# import pandas as pd
# import tkinter as tk
# from tkinter import ttk
# from nba_api.stats.endpoints import leaguedashplayerstats
# import time
# import unicodedata

# # --- CSV laden ---
# df = pd.read_csv('nba_players_2023_2024.csv')
# salary_df = pd.read_csv('player_mit_salary.csv')

# # Sonderzeichen in Spielernamen entfernen
# def normalize_name(name):
#     if pd.isna(name):
#         return name
#     nfkd_form = unicodedata.normalize('NFKD', name)
#     only_ascii = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
#     return only_ascii

# df['DISPLAY_FIRST_LAST'] = df['DISPLAY_FIRST_LAST'].apply(normalize_name)

# # Nur relevante Spalten aus Salary-Datei
# salary_df = salary_df[['PERSON_ID', 'Salary']]

# df['PERSON_ID'] = pd.to_numeric(df['PERSON_ID'], errors='coerce').astype('Int64') # 'Int64' für nullable integer
# salary_df['PERSON_ID'] = pd.to_numeric(salary_df['PERSON_ID'], errors='coerce').astype('Int64')

# # Merge mit Haupt-DataFrame
# df = pd.merge(df, salary_df, on='PERSON_ID', how='left')

# # --- Positionsabhängige Effizienz-Gewichtungen ---
# efficiency_weights = {
#     'Center':            {'PTS': 0.3, 'REB': 0.4, 'AST': 0.05, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.05},
#     'Center-Forward':    {'PTS': 0.3, 'REB': 0.35, 'AST': 0.05, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.1},
#     'Forward':           {'PTS': 0.25, 'REB': 0.2, 'AST': 0.15, 'STL': 0.15, 'BLK': 0.1, 'FG3M': 0.15},
#     'Forward-Center':    {'PTS': 0.3, 'REB': 0.3, 'AST': 0.1, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.1},
#     'Guard':             {'PTS': 0.35, 'REB': 0.1, 'AST': 0.2, 'STL': 0.15, 'BLK': 0.05, 'FG3M': 0.15},
#     'Guard-Forward':     {'PTS': 0.3, 'REB': 0.15, 'AST': 0.2, 'STL': 0.15, 'BLK': 0.05, 'FG3M': 0.15}
# }

# # --- Stats laden ---
# print("Lade aktuelle NBA-Stats ...")
# stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2023-24').get_data_frames()[0]
# time.sleep(1)

# stats = stats[['PLAYER_ID', 'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']]
# stats = stats.rename(columns={'PLAYER_ID': 'PERSON_ID'})
# df = pd.merge(df, stats, on='PERSON_ID', how='left')

# # --- Neue Effizienzberechnung ---
# def calculate_efficiency(row):
#     pos = row['POSITION']
#     weights = efficiency_weights.get(pos, {'PTS': 0.3, 'REB': 0.2, 'AST': 0.15, 'STL': 0.15, 'BLK': 0.1, 'FG3M': 0.1})
#     gp = row['GP']
#     if pd.isna(gp) or gp == 0:
#         return 0
#     eff = 0
#     for stat, weight in weights.items():
#         value = row.get(stat, 0)
#         value = value if pd.notna(value) else 0
#         eff += (value / gp) * weight
#     return eff * 10

# df['EFFICIENCY'] = df.apply(calculate_efficiency, axis=1)

# # --- Spalten ---
# columns = [
#     'DISPLAY_FIRST_LAST', 'POSITION', 'TEAM_NAME', 'Salary',
#     'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY'
# ]

# column_names = {
#     'DISPLAY_FIRST_LAST': 'Name',
#     'POSITION': 'Position',
#     'TEAM_NAME': 'Team',
#     'Salary': 'Jahresgehalt',
#     'GP': 'Spiele',
#     'PTS': 'Punkte',
#     'REB': 'Rebounds',
#     'AST': 'Assists',
#     'STL': 'Steals',
#     'BLK': 'Blocks',
#     'FG3M': '3er Treffer',
#     'EFFICIENCY': 'Effizienz'
# }

# NUMERIC_COL_WIDTH = 80

# text_column_widths = {
#     'DISPLAY_FIRST_LAST': 200,
#     'POSITION': 100,
#     'TEAM_NAME': 150,
#     'Salary': 100
# }

# column_widths = {}
# for col in columns:
#     if col in ['GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY']:
#         column_widths[col] = NUMERIC_COL_WIDTH
#     else:
#         column_widths[col] = text_column_widths.get(col, 100)

# # --- WICHTIG: Effizienz aus den numerischen Filter-Spalten entfernen ---
# numeric_columns = ['GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']

# class EfficiencyApp:
#     def __init__(self, root):
#         self.root = root
#         self.root.title("NBA Spieler Effizienz")
#         self.root.geometry("1300x800")
#         self.root.resizable(width=False, height=True)
#         self.sort_column = None
#         self.sort_reverse = False
#         self.filtered_df = df.copy()

#         self.filters = {col: {'min': '', 'max': ''} for col in numeric_columns + ['EFFICIENCY']}
#         self.show_average = False  # False = Summe, True = Durchschnitt

#         self.create_header()
#         self.create_controls()
#         self.create_filter_row()
#         self.create_table()
#         self.update_table()

#     def create_header(self):
#         header_frame = tk.Frame(self.root, bg="#1D428A", padx=10, pady=10)
#         header_frame.pack(fill="x")

#         tk.Label(header_frame, text="🏀 NBA Spieler Effizienz-Analyse",
#                  font=("Arial", 20, "bold"), fg="white", bg="#1D428A").pack()

#         tk.Label(header_frame, text="🔄 Klicke auf Spaltenüberschriften zum Sortieren",
#                  font=("Arial", 10, "italic"), fg="white", bg="#1D428A").pack()

#     def create_controls(self):
#         control_frame = tk.Frame(self.root, bg="#F0F0F0", padx=10, pady=5)
#         control_frame.pack(fill="x")

#         tk.Label(control_frame, text="🔍 Suche:", font=("Arial", 11), bg="#F0F0F0").pack(side="left", padx=5)
#         self.search_var = tk.StringVar()
#         search_entry = tk.Entry(control_frame, textvariable=self.search_var, font=("Arial", 11), width=30)
#         search_entry.pack(side="left", padx=5)
#         search_entry.bind('<KeyRelease>', self.on_search)

#         self.avg_button = tk.Button(control_frame, text="Durchschnitt anzeigen", command=self.toggle_average)
#         self.avg_button.pack(side="left", padx=10)

#         self.count_label = tk.Label(control_frame, text="", font=("Arial", 10, "bold"), bg="#F0F0F0")
#         self.count_label.pack(side="right", padx=10)

#     def toggle_average(self):
#         self.show_average = not self.show_average
#         if self.show_average:
#             self.avg_button.config(text="Summe anzeigen")
#         else:
#             self.avg_button.config(text="Durchschnitt anzeigen")
#         self.update_table()

#     def create_filter_row(self):
#         self.filter_frame = tk.Frame(self.root)
#         self.filter_frame.pack(fill='x', padx=20)

#         self.filter_entries = {}

#         for col in columns:
#             f = tk.Frame(self.filter_frame, width=column_widths[col], height=40)
#             f.pack_propagate(False)
#             f.pack(side='left', anchor='n')

#             if col in numeric_columns + ['EFFICIENCY']:
#                 min_var = tk.StringVar()
#                 max_var = tk.StringVar()

#                 min_entry = tk.Entry(f, textvariable=min_var, width=6, justify='center', font=('Segoe UI', 8))
#                 max_entry = tk.Entry(f, textvariable=max_var, width=6, justify='center', font=('Segoe UI', 8))

#                 min_entry.insert(0, 'min')
#                 max_entry.insert(0, 'max')

#                 def on_focus_in_min(event, e=min_entry):
#                     if e.get() == 'min':
#                         e.delete(0, 'end')

#                 def on_focus_out_min(event, e=min_entry):
#                     if e.get() == '':
#                         e.insert(0, 'min')

#                 def on_focus_in_max(event, e=max_entry):
#                     if e.get() == 'max':
#                         e.delete(0, 'end')

#                 def on_focus_out_max(event, e=max_entry):
#                     if e.get() == '':
#                         e.insert(0, 'max')

#                 min_entry.bind('<FocusIn>', on_focus_in_min)
#                 min_entry.bind('<FocusOut>', on_focus_out_min)
#                 max_entry.bind('<FocusIn>', on_focus_in_max)
#                 max_entry.bind('<FocusOut>', on_focus_out_max)

#                 min_entry.place(relx=0.5, rely=0.25, anchor='center')
#                 max_entry.place(relx=0.5, rely=0.75, anchor='center')

#                 def on_filter_change(var, indx, mode, col=col, min_var=min_var, max_var=max_var):
#                     self.filters[col]['min'] = min_var.get()
#                     self.filters[col]['max'] = max_var.get()
#                     self.apply_filters()

#                 min_var.trace_add('write', on_filter_change)
#                 max_var.trace_add('write', on_filter_change)

#                 self.filter_entries[col] = {'min': min_entry, 'max': max_entry}
#             else:
#                 tk.Label(f, text='').pack()

#     def create_table(self):
#         table_frame = tk.Frame(self.root)
#         table_frame.pack(fill="both", expand=True, padx=20, pady=10)

#         y_scrollbar = ttk.Scrollbar(table_frame)
#         y_scrollbar.pack(side="right", fill="y")

#         style = ttk.Style()
#         style.theme_use("clam")
#         style.configure("Treeview", background="white", foreground="black",
#                         rowheight=28, fieldbackground="white", font=('Segoe UI', 10))
#         style.configure("Treeview.Heading", font=('Segoe UI', 10, 'bold'),
#                         background="#C8102E", foreground="white")
#         style.map("Treeview.Heading", background=[("active", "#A10E1E")])

#         self.tree = ttk.Treeview(table_frame, columns=columns, show='headings', yscrollcommand=y_scrollbar.set)
#         y_scrollbar.config(command=self.tree.yview)

#         for col in columns:
#             self.tree.heading(col, text=column_names[col], command=lambda c=col: self.sort_by_column(c))
#             self.tree.column(col, width=column_widths[col], anchor='center', stretch=False)

#         self.tree.pack(side="left", fill="both", expand=True)
#         self.tree.tag_configure('odd', background='#F5F5F5')
#         self.tree.tag_configure('even', background='white')

#         def disable_column_resize(event):
#             for col in columns:
#                 self.tree.column(col, width=column_widths[col])

#         self.tree.bind('<ButtonRelease-1>', disable_column_resize)

#     def apply_filters(self):
#         filtered = df.copy()

#         # Suchfilter (Text)
#         search_term = self.search_var.get().lower()
#         if search_term:
#             mask = pd.Series(False, index=filtered.index)
#             for col in columns:
#                 mask |= filtered[col].astype(str).str.lower().str.contains(search_term)
#             filtered = filtered[mask]

#         # Min/Max Filter für numerische Spalten (Summe oder Durchschnitt)
#         for col in numeric_columns + ['EFFICIENCY']:
#             min_val = self.filters.get(col, {}).get('min', '')
#             max_val = self.filters.get(col, {}).get('max', '')

#             def get_filter_value(row):
#                 val = row.get(col, 0)
#                 gp = row.get('GP', 0)
#                 if pd.isna(val) or pd.isna(gp) or gp == 0:
#                     return 0
#                 if col == 'EFFICIENCY':
#                     # Effizienz immer als Originalwert für Filter (nicht Durchschnitt)
#                     return round(val, 1)
#                 if self.show_average and col != 'GP':
#                     return round(val / gp, 1)
#                 return round(val, 1)

#             try:
#                 if min_val not in ['', 'min']:
#                     min_val_f = float(min_val)
#                     filtered = filtered[filtered.apply(lambda r: get_filter_value(r) >= min_val_f, axis=1)]
#                 if max_val not in ['', 'max']:
#                     max_val_f = float(max_val)
#                     filtered = filtered[filtered.apply(lambda r: get_filter_value(r) <= max_val_f, axis=1)]
#             except ValueError:
#                 pass  # ungültige Eingabe ignorieren

#         self.filtered_df = filtered
#         self.update_table()

#     def update_table(self):
#         for row in self.tree.get_children():
#             self.tree.delete(row)

#         df_display = self.filtered_df.copy()

#         # Durchschnittswerte berechnen, EFFICIENCY bleibt unverändert
#         if self.show_average:
#             for stat in ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']:
#                 df_display[stat] = df_display.apply(lambda r: round((r[stat] / r['GP']) if r['GP'] > 0 else 0, 1), axis=1)
#             # EFFICIENCY bleibt original

#         # Sortierung
#         if self.sort_column:
#             ascending = not self.sort_reverse
#             df_display = df_display.sort_values(by=self.sort_column, ascending=ascending)

#         # Anzeige vorbereiten
#         for i, row in enumerate(df_display.itertuples()):
#             vals = []
#             for col in columns:
#                 val = getattr(row, col)
#                 if col == 'Salary' and pd.notna(val):
#                     try:
#                         val = f"${float(val):,.0f}"
#                     except Exception:
#                         val = val
#                 elif col in ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY']:
#                     if pd.isna(val):
#                         val = ''
#                     else:
#                         val = round(val, 1) if col == 'EFFICIENCY' else int(round(val))
#                 vals.append(val)
#             tag = 'even' if i % 2 == 0 else 'odd'
#             self.tree.insert('', 'end', values=vals, tags=(tag,))

#         self.count_label.config(text=f"Gefundene Spieler: {len(df_display)}")

#     def sort_by_column(self, col):
#         if self.sort_column == col:
#             self.sort_reverse = not self.sort_reverse
#         else:
#             self.sort_column = col
#             self.sort_reverse = False
#         self.update_table()

#     def on_search(self, event):
#         self.apply_filters()

# # --- Start GUI ---
# root = tk.Tk()
# app = EfficiencyApp(root)
# root.mainloop()


ParserError: Error tokenizing data. C error: Expected 2 fields in line 16, saw 4


In [ ]:
# import pandas as pd
# import tkinter as tk
# from tkinter import ttk
# from nba_api.stats.endpoints import leaguedashplayerstats
# import time
# import unicodedata

# # --- CSV laden ---
# df = pd.read_csv('nba_players_2023_2024.csv')
# salary_df = pd.read_csv('player_mit_salary.csv', sep=";")

# print("\n--- Initialer Lade-Check ---")
# print("Anzahl Spieler in df (nba_players_2023_2024):", len(df))
# print("Anzahl Spieler in salary_df (player_mit_salary):", len(salary_df))
# print("Erste 3 Zeilen von salary_df:")
# print(salary_df.head(3))
# print("Datentypen in salary_df vor Anpassung:")
# print(salary_df.dtypes)


# # Sonderzeichen in Spielernamen entfernen
# def normalize_name(name):
#     if pd.isna(name):
#         return name
#     nfkd_form = unicodedata.normalize('NFKD', name)
#     only_ascii = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
#     return only_ascii

# df['DISPLAY_FIRST_LAST'] = df['DISPLAY_FIRST_LAST'].apply(normalize_name)

# # Nur relevante Spalten aus Salary-Datei
# # Stellen Sie sicher, dass 'PERSON_ID' und 'Salary' existieren
# if 'PERSON_ID' not in salary_df.columns or 'Salary' not in salary_df.columns:
#     print("\nFEHLER: 'PERSON_ID' oder 'Salary' Spalte nicht in player_mit_salary.csv gefunden!")
#     # Hier können Sie das Skript beenden oder einen Fehlerbehandlungsmechanismus einbauen
#     exit() # Für Debugging-Zwecke

# salary_df = salary_df[['PERSON_ID', 'Salary']]

# # Datentypen anpassen
# df['PERSON_ID'] = pd.to_numeric(df['PERSON_ID'], errors='coerce').astype('Int64') # 'Int64' für nullable integer
# # Wichtig: Konvertieren Sie auch die Salary-Spalte zu einem numerischen Typ!
# salary_df['PERSON_ID'] = pd.to_numeric(salary_df['PERSON_ID'], errors='coerce').astype('Int64')
# salary_df['Salary'] = pd.to_numeric(salary_df['Salary'], errors='coerce') # Konvertiere Gehalt zu float

# print("\n--- Datentypen nach Anpassung vor Merge ---")
# print("Datentyp von df['PERSON_ID']:", df['PERSON_ID'].dtype)
# print("Datentyp von salary_df['PERSON_ID']:", salary_df['PERSON_ID'].dtype)
# print("Datentyp von salary_df['Salary']:", salary_df['Salary'].dtype)
# print("Beispielhafte PERSON_ID's in df:", df['PERSON_ID'].head(5).tolist())
# print("Beispielhafte PERSON_ID's in salary_df:", salary_df['PERSON_ID'].head(5).tolist())


# # Merge mit Haupt-DataFrame
# df = pd.merge(df, salary_df, on='PERSON_ID', how='left')

# print("\n--- Nach dem Merge ---")
# print("Anzahl Zeilen im gemergten df:", len(df))
# print("Anzahl Spieler mit Gehalt (nicht-NaN):", df['Salary'].count())
# print("Erste 5 Zeilen des gemergten df mit Salary:")
# print(df[['DISPLAY_FIRST_LAST', 'PERSON_ID', 'Salary']].head(5))


# # ... (Rest Ihres Codes bleibt gleich)
# # --- Positionsabhängige Effizienz-Gewichtungen ---
# efficiency_weights = {
#     'Center':            {'PTS': 0.3, 'REB': 0.4, 'AST': 0.05, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.05},
#     'Center-Forward':    {'PTS': 0.3, 'REB': 0.35, 'AST': 0.05, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.1},
#     'Forward':           {'PTS': 0.25, 'REB': 0.2, 'AST': 0.15, 'STL': 0.15, 'BLK': 0.1, 'FG3M': 0.15},
#     'Forward-Center':    {'PTS': 0.3, 'REB': 0.3, 'AST': 0.1, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.1},
#     'Guard':             {'PTS': 0.35, 'REB': 0.1, 'AST': 0.2, 'STL': 0.15, 'BLK': 0.05, 'FG3M': 0.15},
#     'Guard-Forward':     {'PTS': 0.3, 'REB': 0.15, 'AST': 0.2, 'STL': 0.15, 'BLK': 0.05, 'FG3M': 0.15}
# }

# # --- Stats laden ---
# print("\nLade aktuelle NBA-Stats ...")
# stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2023-24').get_data_frames()[0]
# time.sleep(1)

# stats = stats[['PLAYER_ID', 'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']]
# stats = stats.rename(columns={'PLAYER_ID': 'PERSON_ID'})
# df = pd.merge(df, stats, on='PERSON_ID', how='left')

# # ... (restlicher Code)
# # Stellen Sie sicher, dass Sie den Rest des Codes nicht vergessen haben!
# # Die Klasse EfficiencyApp und deren Methoden sollten unverändert bleiben,
# # es sei denn, Sie haben dort ebenfalls Änderungen vorgenommen, die das Problem verursachen könnten.

# # --- Neue Effizienzberechnung ---
# def calculate_efficiency(row):
#     pos = row['POSITION']
#     weights = efficiency_weights.get(pos, {'PTS': 0.3, 'REB': 0.2, 'AST': 0.15, 'STL': 0.15, 'BLK': 0.1, 'FG3M': 0.1})
#     gp = row['GP']
#     if pd.isna(gp) or gp == 0:
#         return 0
#     eff = 0
#     for stat, weight in weights.items():
#         value = row.get(stat, 0)
#         value = value if pd.notna(value) else 0
#         eff += (value / gp) * weight
#     return eff * 10

# df['EFFICIENCY'] = df.apply(calculate_efficiency, axis=1)

# # --- Spalten ---
# columns = [
#     'DISPLAY_FIRST_LAST', 'POSITION', 'TEAM_NAME', 'Salary',
#     'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY'
# ]

# column_names = {
#     'DISPLAY_FIRST_LAST': 'Name',
#     'POSITION': 'Position',
#     'TEAM_NAME': 'Team',
#     'Salary': 'Jahresgehalt',
#     'GP': 'Spiele',
#     'PTS': 'Punkte',
#     'REB': 'Rebounds',
#     'AST': 'Assists',
#     'STL': 'Steals',
#     'BLK': 'Blocks',
#     'FG3M': '3er Treffer',
#     'EFFICIENCY': 'Effizienz'
# }

# NUMERIC_COL_WIDTH = 80

# text_column_widths = {
#     'DISPLAY_FIRST_LAST': 200,
#     'POSITION': 100,
#     'TEAM_NAME': 150,
#     'Salary': 100
# }

# column_widths = {}
# for col in columns:
#     if col in ['GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY']:
#         column_widths[col] = NUMERIC_COL_WIDTH
#     else:
#         column_widths[col] = text_column_widths.get(col, 100)

# # --- WICHTIG: Effizienz aus den numerischen Filter-Spalten entfernen ---
# numeric_columns = ['GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']

# class EfficiencyApp:
#     def __init__(self, root):
#         self.root = root
#         self.root.title("NBA Spieler Effizienz")
#         self.root.geometry("1300x800")
#         self.root.resizable(width=False, height=True)
#         self.sort_column = None
#         self.sort_reverse = False
#         self.filtered_df = df.copy()

#         self.filters = {col: {'min': '', 'max': ''} for col in numeric_columns + ['EFFICIENCY']}
#         self.show_average = False  # False = Summe, True = Durchschnitt

#         self.create_header()
#         self.create_controls()
#         self.create_filter_row()
#         self.create_table()
#         self.update_table()

#     def create_header(self):
#         header_frame = tk.Frame(self.root, bg="#1D428A", padx=10, pady=10)
#         header_frame.pack(fill="x")

#         tk.Label(header_frame, text="🏀 NBA Spieler Effizienz-Analyse",
#                  font=("Arial", 20, "bold"), fg="white", bg="#1D428A").pack()

#         tk.Label(header_frame, text="🔄 Klicke auf Spaltenüberschriften zum Sortieren",
#                  font=("Arial", 10, "italic"), fg="white", bg="#1D428A").pack()

#     def create_controls(self):
#         control_frame = tk.Frame(self.root, bg="#F0F0F0", padx=10, pady=5)
#         control_frame.pack(fill="x")

#         tk.Label(control_frame, text="🔍 Suche:", font=("Arial", 11), bg="#F0F0F0").pack(side="left", padx=5)
#         self.search_var = tk.StringVar()
#         search_entry = tk.Entry(control_frame, textvariable=self.search_var, font=("Arial", 11), width=30)
#         search_entry.pack(side="left", padx=5)
#         search_entry.bind('<KeyRelease>', self.on_search)

#         self.avg_button = tk.Button(control_frame, text="Durchschnitt anzeigen", command=self.toggle_average)
#         self.avg_button.pack(side="left", padx=10)

#         self.count_label = tk.Label(control_frame, text="", font=("Arial", 10, "bold"), bg="#F0F0F0")
#         self.count_label.pack(side="right", padx=10)

#     def toggle_average(self):
#         self.show_average = not self.show_average
#         if self.show_average:
#             self.avg_button.config(text="Summe anzeigen")
#         else:
#             self.avg_button.config(text="Durchschnitt anzeigen")
#         self.update_table()

#     def create_filter_row(self):
#         self.filter_frame = tk.Frame(self.root)
#         self.filter_frame.pack(fill='x', padx=20)

#         self.filter_entries = {}

#         for col in columns:
#             f = tk.Frame(self.filter_frame, width=column_widths[col], height=40)
#             f.pack_propagate(False)
#             f.pack(side='left', anchor='n')

#             if col in numeric_columns + ['EFFICIENCY']:
#                 min_var = tk.StringVar()
#                 max_var = tk.StringVar()

#                 min_entry = tk.Entry(f, textvariable=min_var, width=6, justify='center', font=('Segoe UI', 8))
#                 max_entry = tk.Entry(f, textvariable=max_var, width=6, justify='center', font=('Segoe UI', 8))

#                 min_entry.insert(0, 'min')
#                 max_entry.insert(0, 'max')

#                 def on_focus_in_min(event, e=min_entry):
#                     if e.get() == 'min':
#                         e.delete(0, 'end')

#                 def on_focus_out_min(event, e=min_entry):
#                     if e.get() == '':
#                         e.insert(0, 'min')

#                 def on_focus_in_max(event, e=max_entry):
#                     if e.get() == 'max':
#                         e.delete(0, 'end')

#                 def on_focus_out_max(event, e=max_entry):
#                     if e.get() == '':
#                         e.insert(0, 'max')

#                 min_entry.bind('<FocusIn>', on_focus_in_min)
#                 min_entry.bind('<FocusOut>', on_focus_out_min)
#                 max_entry.bind('<FocusIn>', on_focus_in_max)
#                 max_entry.bind('<FocusOut>', on_focus_out_max)

#                 min_entry.place(relx=0.5, rely=0.25, anchor='center')
#                 max_entry.place(relx=0.5, rely=0.75, anchor='center')

#                 def on_filter_change(var, indx, mode, col=col, min_var=min_var, max_var=max_var):
#                     self.filters[col]['min'] = min_var.get()
#                     self.filters[col]['max'] = max_var.get()
#                     self.apply_filters()

#                 min_var.trace_add('write', on_filter_change)
#                 max_var.trace_add('write', on_filter_change)

#                 self.filter_entries[col] = {'min': min_entry, 'max': max_entry}
#             else:
#                 tk.Label(f, text='').pack()

#     def create_table(self):
#         table_frame = tk.Frame(self.root)
#         table_frame.pack(fill="both", expand=True, padx=20, pady=10)

#         y_scrollbar = ttk.Scrollbar(table_frame)
#         y_scrollbar.pack(side="right", fill="y")

#         style = ttk.Style()
#         style.theme_use("clam")
#         style.configure("Treeview", background="white", foreground="black",
#                         rowheight=28, fieldbackground="white", font=('Segoe UI', 10))
#         style.configure("Treeview.Heading", font=('Segoe UI', 10, 'bold'),
#                         background="#C8102E", foreground="white")
#         style.map("Treeview.Heading", background=[("active", "#A10E1E")])

#         self.tree = ttk.Treeview(table_frame, columns=columns, show='headings', yscrollcommand=y_scrollbar.set)
#         y_scrollbar.config(command=self.tree.yview)

#         for col in columns:
#             self.tree.heading(col, text=column_names[col], command=lambda c=col: self.sort_by_column(c))
#             self.tree.column(col, width=column_widths[col], anchor='center', stretch=False)

#         self.tree.pack(side="left", fill="both", expand=True)
#         self.tree.tag_configure('odd', background='#F5F5F5')
#         self.tree.tag_configure('even', background='white')

#         def disable_column_resize(event):
#             for col in columns:
#                 self.tree.column(col, width=column_widths[col])

#         self.tree.bind('<ButtonRelease-1>', disable_column_resize)

#     def apply_filters(self):
#         filtered = df.copy()

#         # Suchfilter (Text)
#         search_term = self.search_var.get().lower()
#         if search_term:
#             mask = pd.Series(False, index=filtered.index)
#             for col in columns:
#                 # Sicherstellen, dass die Spalte vorhanden ist und zu String konvertiert werden kann
#                 if col in filtered.columns:
#                     mask |= filtered[col].astype(str).str.lower().str.contains(search_term, na=False)
#             filtered = filtered[mask]

#         # Min/Max Filter für numerische Spalten (Summe oder Durchschnitt)
#         for col in numeric_columns + ['EFFICIENCY']:
#             min_val = self.filters.get(col, {}).get('min', '')
#             max_val = self.filters.get(col, {}).get('max', '')

#             def get_filter_value(row):
#                 val = row.get(col, 0)
#                 gp = row.get('GP', 0)
#                 if pd.isna(val) or pd.isna(gp) or gp == 0:
#                     return 0
#                 if col == 'EFFICIENCY':
#                     # Effizienz immer als Originalwert für Filter (nicht Durchschnitt)
#                     return round(val, 1)
#                 if self.show_average and col != 'GP':
#                     return round(val / gp, 1)
#                 return round(val, 1)

#             try:
#                 if min_val not in ['', 'min']:
#                     min_val_f = float(min_val)
#                     filtered = filtered[filtered.apply(lambda r: get_filter_value(r) >= min_val_f, axis=1)]
#                 if max_val not in ['', 'max']:
#                     max_val_f = float(max_val)
#                     filtered = filtered[filtered.apply(lambda r: get_filter_value(r) <= max_val_f, axis=1)]
#             except ValueError:
#                 pass  # ungültige Eingabe ignorieren

#         self.filtered_df = filtered
#         self.update_table()

#     def update_table(self):
#         for row in self.tree.get_children():
#             self.tree.delete(row)

#         df_display = self.filtered_df.copy()

#         # Durchschnittswerte berechnen, EFFICIENCY bleibt unverändert
#         if self.show_average:
#             for stat in ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']:
#                 df_display[stat] = df_display.apply(lambda r: round((r[stat] / r['GP']) if pd.notna(r[stat]) and pd.notna(r['GP']) and r['GP'] > 0 else 0, 1), axis=1)
#             # EFFICIENCY bleibt original

#         # Sortierung
#         if self.sort_column:
#             ascending = not self.sort_reverse
#             # Beim Sortieren numerischer Spalten mit NaN-Werten diese an das Ende setzen
#             if df_display[self.sort_column].dtype in ['float64', 'Int64']:
#                 df_display = df_display.sort_values(
#                     by=self.sort_column,
#                     ascending=ascending,
#                     na_position='last' # NaN-Werte am Ende
#                 )
#             else: # Für Textspalten
#                 df_display = df_display.sort_values(by=self.sort_column, ascending=ascending)


#         # Anzeige vorbereiten
#         for i, row in enumerate(df_display.itertuples()):
#             vals = []
#             for col in columns:
#                 val = getattr(row, col, None) # Sicherstellen, dass getattr einen Default-Wert hat
#                 if col == 'Salary' and pd.notna(val):
#                     try:
#                         val = f"${float(val):,.0f}"
#                     except Exception:
#                         val = val # Wenn die Konvertierung fehlschlägt, behalte den Wert
#                 elif col in ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY']:
#                     if pd.isna(val):
#                         val = ''
#                     else:
#                         # Achten Sie darauf, dass 'GP' auch gerundet wird, wenn es als numerische Statistik angezeigt wird
#                         if col == 'GP':
#                             val = int(round(val))
#                         elif col == 'EFFICIENCY':
#                             val = round(val, 1)
#                         else:
#                             val = round(val, 1) # Runden auf 1 Dezimalstelle für Statistiken
#                 vals.append(val)
#             tag = 'even' if i % 2 == 0 else 'odd'
#             self.tree.insert('', 'end', values=vals, tags=(tag,))

#         self.count_label.config(text=f"Gefundene Spieler: {len(df_display)}")

#     def sort_by_column(self, col):
#         if self.sort_column == col:
#             self.sort_reverse = not self.sort_reverse
#         else:
#             self.sort_column = col
#             self.sort_reverse = False
#         self.update_table()

#     def on_search(self, event):
#         self.apply_filters()

# # --- Start GUI ---
# root = tk.Tk()
# app = EfficiencyApp(root)
# root.mainloop()


--- Initialer Lade-Check ---
Anzahl Spieler in df (nba_players_2023_2024): 572
Anzahl Spieler in salary_df (player_mit_salary): 573
Erste 3 Zeilen von salary_df:
   PERSON_ID FIRST_NAME LAST_NAME DISPLAY_FIRST_LAST DISPLAY_LAST_COMMA_FIRST  \
0    1630173   Precious   Achiuwa   Precious Achiuwa        Achiuwa, Precious   
1     203500     Steven     Adams       Steven Adams            Adams, Steven   
2    1628389        Bam   Adebayo        Bam Adebayo             Adebayo, Bam   

  DISPLAY_FI_LAST       PLAYER_SLUG            BIRTHDATE      SCHOOL  \
0      P. Achiuwa  precious-achiuwa  1999-09-19T00:00:00     Memphis   
1        S. Adams      steven-adams  1993-07-20T00:00:00  Pittsburgh   
2      B. Adebayo       bam-adebayo  1997-07-18T00:00:00    Kentucky   

       COUNTRY  ... TO_YEAR DLEAGUE_FLAG  NBA_FLAG  GAMES_PLAYED_FLAG  \
0      Nigeria  ...    2024            N         Y                  Y   
1  New Zealand  ...    2024            N         Y                  Y   
2   

In [2]:
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from nba_api.stats.endpoints import leaguedashplayerstats
import time
import unicodedata
import json # Importieren der JSON-Bibliothek

# --- CSV laden ---
df = pd.read_csv('nba_players_2023_2024.csv')
salary_df = pd.read_csv('player_mit_salary.csv', sep=";")

print("\n--- Initialer Lade-Check ---")
print("Anzahl Spieler in df (nba_players_2023_2024):", len(df))
print("Anzahl Spieler in salary_df (player_mit_salary):", len(salary_df))
print("Erste 3 Zeilen von salary_df:")
print(salary_df.head(3))
print("Datentypen in salary_df vor Anpassung:")
print(salary_df.dtypes)

# Sonderzeichen in Spielernamen entfernen
def normalize_name(name):
    if pd.isna(name):
        return name
    nfkd_form = unicodedata.normalize('NFKD', name)
    only_ascii = nfkd_form.encode('ASCII', 'ignore').decode('ASCII')
    return only_ascii

df['DISPLAY_FIRST_LAST'] = df['DISPLAY_FIRST_LAST'].apply(normalize_name)

# Nur relevante Spalten aus Salary-Datei
# Stellen Sie sicher, dass 'PERSON_ID' und 'Salary' existieren
if 'PERSON_ID' not in salary_df.columns or 'Salary' not in salary_df.columns:
    print("\nFEHLER: 'PERSON_ID' oder 'Salary' Spalte nicht in player_mit_salary.csv gefunden!")
    exit() # Für Debugging-Zwecke

salary_df = salary_df[['PERSON_ID', 'Salary']]

# Datentypen anpassen
df['PERSON_ID'] = pd.to_numeric(df['PERSON_ID'], errors='coerce').astype('Int64') # 'Int64' für nullable integer
# Wichtig: Konvertieren Sie auch die Salary-Spalte zu einem numerischen Typ!
salary_df['PERSON_ID'] = pd.to_numeric(salary_df['PERSON_ID'], errors='coerce').astype('Int64')
salary_df['Salary'] = pd.to_numeric(salary_df['Salary'], errors='coerce') # Konvertiere Gehalt zu float

print("\n--- Datentypen nach Anpassung vor Merge ---")
print("Datentyp von df['PERSON_ID']:", df['PERSON_ID'].dtype)
print("Datentyp von salary_df['PERSON_ID']:", salary_df['PERSON_ID'].dtype)
print("Datentyp von salary_df['Salary']:", salary_df['Salary'].dtype)
print("Beispielhafte PERSON_ID's in df:", salary_df['PERSON_ID'].head(5).tolist())
print("Beispielhafte PERSON_ID's in salary_df:", salary_df['PERSON_ID'].head(5).tolist())


# Merge mit Haupt-DataFrame
df = pd.merge(df, salary_df, on='PERSON_ID', how='left')

print("\n--- Nach dem Merge ---")
print("Anzahl Zeilen im gemergten df:", len(df))
print("Anzahl Spieler mit Gehalt (nicht-NaN):", df['Salary'].count())
print("Erste 5 Zeilen des gemergten df mit Salary:")
print(df[['DISPLAY_FIRST_LAST', 'PERSON_ID', 'Salary']].head(5))

# --- Positionsabhängige Effizienz-Gewichtungen ---
efficiency_weights = {
    'Center':               {'PTS': 0.3, 'REB': 0.4, 'AST': 0.05, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.05},
    'Center-Forward':       {'PTS': 0.3, 'REB': 0.35, 'AST': 0.05, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.1},
    'Forward':              {'PTS': 0.25, 'REB': 0.2, 'AST': 0.15, 'STL': 0.15, 'BLK': 0.1, 'FG3M': 0.15},
    'Forward-Center':       {'PTS': 0.3, 'REB': 0.3, 'AST': 0.1, 'STL': 0.1, 'BLK': 0.1, 'FG3M': 0.1},
    'Guard':                {'PTS': 0.35, 'REB': 0.1, 'AST': 0.2, 'STL': 0.15, 'BLK': 0.05, 'FG3M': 0.15},
    'Guard-Forward':        {'PTS': 0.3, 'REB': 0.15, 'AST': 0.2, 'STL': 0.15, 'BLK': 0.05, 'FG3M': 0.15}
}

# --- Stats laden ---
print("\nLade aktuelle NBA-Stats ...")
stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2023-24').get_data_frames()[0]
time.sleep(1)

stats = stats[['PLAYER_ID', 'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']]
stats = stats.rename(columns={'PLAYER_ID': 'PERSON_ID'})
df = pd.merge(df, stats, on='PERSON_ID', how='left')

# --- WICHTIG: Duplikate nach PERSON_ID entfernen, um TclError zu vermeiden ---
# Dies stellt sicher, dass jede PERSON_ID im DataFrame einzigartig ist,
# bevor sie als Treeview iid verwendet wird.
df.drop_duplicates(subset=['PERSON_ID'], inplace=True)
print("\n--- Nach Duplikatsbereinigung nach PERSON_ID ---")
print("Anzahl Zeilen im df nach Duplikatsbereinigung:", len(df))


# --- Neue Effizienzberechnung ---
def calculate_efficiency(row):
    pos = row['POSITION']
    weights = efficiency_weights.get(pos, {'PTS': 0.3, 'REB': 0.2, 'AST': 0.15, 'STL': 0.15, 'BLK': 0.1, 'FG3M': 0.1})
    gp = row['GP']
    if pd.isna(gp) or gp == 0:
        return 0
    eff = 0
    for stat, weight in weights.items():
        value = row.get(stat, 0)
        value = value if pd.notna(value) else 0
        eff += (value / gp) * weight
    return eff * 10

df['EFFICIENCY'] = df.apply(calculate_efficiency, axis=1)

# --- Spalten ---
columns = [
    'DISPLAY_FIRST_LAST', 'POSITION', 'TEAM_NAME', 'Salary',
    'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY'
]

column_names = {
    'DISPLAY_FIRST_LAST': 'Name',
    'POSITION': 'Position',
    'TEAM_NAME': 'Team',
    'Salary': 'Jahresgehalt',
    'GP': 'Spiele',
    'PTS': 'Punkte',
    'REB': 'Rebounds',
    'AST': 'Assists',
    'STL': 'Steals',
    'BLK': 'Blocks',
    'FG3M': '3er Treffer',
    'EFFICIENCY': 'Effizienz'
}

NUMERIC_COL_WIDTH = 80

text_column_widths = {
    'DISPLAY_FIRST_LAST': 200,
    'POSITION': 100,
    'TEAM_NAME': 150,
    'Salary': 100
}

column_widths = {}
for col in columns:
    if col in ['GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY']:
        column_widths[col] = NUMERIC_COL_WIDTH
    else:
        column_widths[col] = text_column_widths.get(col, 100)

# --- WICHTIG: Effizienz aus den numerischen Filter-Spalten entfernen ---
numeric_columns = ['GP', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']

class EfficiencyApp:
    def __init__(self, root):
        self.root = root
        self.root.title("NBA Spieler Effizienz")
        self.root.geometry("1300x800")
        self.root.resizable(width=False, height=True)
        self.sort_column = None
        self.sort_reverse = False
        self.filtered_df = df.copy() # Initial ist filtered_df eine Kopie des gesamten df

        self.filters = {col: {'min': '', 'max': ''} for col in numeric_columns + ['EFFICIENCY']}
        self.show_average = False   # False = Summe, True = Durchschnitt

        self.create_header()
        self.create_controls()
        self.create_filter_row()
        self.create_table()
        self.update_table()

    def create_header(self):
        header_frame = tk.Frame(self.root, bg="#1D428A", padx=10, pady=10)
        header_frame.pack(fill="x")

        tk.Label(header_frame, text="🏀 NBA Spieler Effizienz-Analyse",
                 font=("Arial", 20, "bold"), fg="white", bg="#1D428A").pack()

        tk.Label(header_frame, text="🔄 Klicke auf Spaltenüberschriften zum Sortieren",
                 font=("Arial", 10, "italic"), fg="white", bg="#1D428A").pack()

    def create_controls(self):
        control_frame = tk.Frame(self.root, bg="#F0F0F0", padx=10, pady=5)
        control_frame.pack(fill="x")

        tk.Label(control_frame, text="🔍 Suche:", font=("Arial", 11), bg="#F0F0F0").pack(side="left", padx=5)
        self.search_var = tk.StringVar()
        search_entry = tk.Entry(control_frame, textvariable=self.search_var, font=("Arial", 11), width=30)
        search_entry.pack(side="left", padx=5)
        search_entry.bind('<KeyRelease>', self.on_search)

        self.avg_button = tk.Button(control_frame, text="Durchschnitt anzeigen", command=self.toggle_average)
        self.avg_button.pack(side="left", padx=10)

        # Neuer Export Button
        export_button = tk.Button(control_frame, text="Ausgewählte Spieler-Namen exportieren", command=self.export_selected_players_as_json)
        export_button.pack(side="left", padx=10)

        self.count_label = tk.Label(control_frame, text="", font=("Arial", 10, "bold"), bg="#F0F0F0")
        self.count_label.pack(side="right", padx=10)

    def toggle_average(self):
        self.show_average = not self.show_average
        if self.show_average:
            self.avg_button.config(text="Summe anzeigen")
        else:
            self.avg_button.config(text="Durchschnitt anzeigen")
        self.update_table()

    def create_filter_row(self):
        self.filter_frame = tk.Frame(self.root)
        self.filter_frame.pack(fill='x', padx=20)

        self.filter_entries = {}

        for col in columns:
            f = tk.Frame(self.filter_frame, width=column_widths[col], height=40)
            f.pack_propagate(False)
            f.pack(side='left', anchor='n')

            if col in numeric_columns + ['EFFICIENCY']:
                min_var = tk.StringVar()
                max_var = tk.StringVar()

                min_entry = tk.Entry(f, textvariable=min_var, width=6, justify='center', font=('Segoe UI', 8))
                max_entry = tk.Entry(f, textvariable=max_var, width=6, justify='center', font=('Segoe UI', 8))

                min_entry.insert(0, 'min')
                max_entry.insert(0, 'max')

                def on_focus_in_min(event, e=min_entry):
                    if e.get() == 'min':
                        e.delete(0, 'end')

                def on_focus_out_min(event, e=min_entry):
                    if e.get() == '':
                        e.insert(0, 'min')

                def on_focus_in_max(event, e=max_entry):
                    if e.get() == 'max':
                        e.delete(0, 'end')

                def on_focus_out_max(event, e=max_entry):
                    if e.get() == '':
                        e.insert(0, 'max')

                min_entry.bind('<FocusIn>', on_focus_in_min)
                min_entry.bind('<FocusOut>', on_focus_out_min)
                max_entry.bind('<FocusIn>', on_focus_in_max)
                max_entry.bind('<FocusOut>', on_focus_out_max)

                min_entry.place(relx=0.5, rely=0.25, anchor='center')
                max_entry.place(relx=0.5, rely=0.75, anchor='center')

                def on_filter_change(var, indx, mode, col=col, min_var=min_var, max_var=max_var):
                    self.filters[col]['min'] = min_var.get()
                    self.filters[col]['max'] = max_var.get()
                    self.apply_filters()

                min_var.trace_add('write', on_filter_change)
                max_var.trace_add('write', on_filter_change)

                self.filter_entries[col] = {'min': min_entry, 'max': max_entry}
            else:
                tk.Label(f, text='').pack()

    def create_table(self):
        table_frame = tk.Frame(self.root)
        table_frame.pack(fill="both", expand=True, padx=20, pady=10)

        y_scrollbar = ttk.Scrollbar(table_frame)
        y_scrollbar.pack(side="right", fill="y")

        style = ttk.Style()
        style.theme_use("clam")
        style.configure("Treeview", background="white", foreground="black",
                        rowheight=28, fieldbackground="white", font=('Segoe UI', 10))
        style.configure("Treeview.Heading", font=('Segoe UI', 10, 'bold'),
                        background="#C8102E", foreground="white")
        style.map("Treeview.Heading", background=[("active", "#A10E1E")])

        self.tree = ttk.Treeview(table_frame, columns=columns, show='headings', yscrollcommand=y_scrollbar.set)
        y_scrollbar.config(command=self.tree.yview)

        for col in columns:
            self.tree.heading(col, text=column_names[col], command=lambda c=col: self.sort_by_column(c))
            self.tree.column(col, width=column_widths[col], anchor='center', stretch=False)

        self.tree.pack(side="left", fill="both", expand=True)
        self.tree.tag_configure('odd', background='#F5F5F5')
        self.tree.tag_configure('even', background='white')

        def disable_column_resize(event):
            for col in columns:
                self.tree.column(col, width=column_widths[col])

        self.tree.bind('<ButtonRelease-1>', disable_column_resize)

    def apply_filters(self):
        filtered = df.copy()

        # Suchfilter (Text)
        search_term = self.search_var.get().lower()
        if search_term:
            mask = pd.Series(False, index=filtered.index)
            for col in columns:
                # Sicherstellen, dass die Spalte vorhanden ist und zu String konvertiert werden kann
                if col in filtered.columns:
                    # 'astype(str)' und 'na=False' handhaben fehlende Werte robust
                    mask |= filtered[col].astype(str).str.lower().str.contains(search_term, na=False)
            filtered = filtered[mask]

        # Min/Max Filter für numerische Spalten (Summe oder Durchschnitt)
        for col in numeric_columns + ['EFFICIENCY']:
            min_val = self.filters.get(col, {}).get('min', '')
            max_val = self.filters.get(col, {}).get('max', '')

            def get_filter_value(row):
                val = row.get(col, 0)
                gp = row.get('GP', 0)
                if pd.isna(val) or pd.isna(gp) or gp == 0:
                    return 0
                if col == 'EFFICIENCY':
                    # Effizienz immer als Originalwert für Filter (nicht Durchschnitt)
                    return round(val, 1)
                if self.show_average and col != 'GP': # 'GP' wird nie als Durchschnitt angezeigt, nur als Summe
                    return round(val / gp, 1)
                return round(val, 1)

            try:
                if min_val not in ['', 'min']:
                    min_val_f = float(min_val)
                    # apply() auf jeder Zeile, um den gefilterten Wert zu erhalten
                    filtered = filtered[filtered.apply(lambda r: get_filter_value(r) >= min_val_f, axis=1)]
                if max_val not in ['', 'max']:
                    max_val_f = float(max_val)
                    filtered = filtered[filtered.apply(lambda r: get_filter_value(r) <= max_val_f, axis=1)]
            except ValueError:
                pass  # ungültige Eingabe ignorieren

        self.filtered_df = filtered
        self.update_table()

    def update_table(self):
        for row in self.tree.get_children():
            self.tree.delete(row)

        df_display = self.filtered_df.copy()

        # Durchschnittswerte berechnen, EFFICIENCY bleibt unverändert
        if self.show_average:
            for stat in ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M']:
                # Stellen Sie sicher, dass die Berechnung nur durchgeführt wird, wenn GP > 0
                df_display[stat] = df_display.apply(
                    lambda r: round((r[stat] / r['GP']) if pd.notna(r[stat]) and pd.notna(r['GP']) and r['GP'] > 0 else 0, 1),
                    axis=1
                )
            # EFFICIENCY bleibt original
        
        # Sortierung
        if self.sort_column:
            ascending = not self.sort_reverse
            # Beim Sortieren numerischer Spalten mit NaN-Werten diese an das Ende setzen
            if df_display[self.sort_column].dtype in ['float64', 'Int64']:
                df_display = df_display.sort_values(
                    by=self.sort_column,
                    ascending=ascending,
                    na_position='last' # NaN-Werte am Ende
                )
            else: # Für Textspalten
                df_display = df_display.sort_values(by=self.sort_column, ascending=ascending)

        # Anzeige vorbereiten
        for i, row in enumerate(df_display.itertuples()):
            vals = []
            for col in columns:
                val = getattr(row, col, None) # Sicherstellen, dass getattr einen Default-Wert hat
                if col == 'Salary' and pd.notna(val):
                    try:
                        val = f"${float(val):,.0f}"
                    except Exception:
                        val = val # Wenn die Konvertierung fehlschlägt, behalte den Wert
                elif col in ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FG3M', 'EFFICIENCY']:
                    if pd.isna(val):
                        val = ''
                    else:
                        # Achten Sie darauf, dass 'GP' auch gerundet wird, wenn es als numerische Statistik angezeigt wird
                        if col == 'GP':
                            val = int(round(val))
                        elif col == 'EFFICIENCY':
                            val = round(val, 1)
                        else:
                            val = round(val, 1) # Runden auf 1 Dezimalstelle für Statistiken
                vals.append(val)
            tag = 'even' if i % 2 == 0 else 'odd'
            # Verwende PERSON_ID als iid, um ausgewählte Spieler später eindeutig zu identifizieren
            self.tree.insert('', 'end', iid=row.PERSON_ID, values=vals, tags=(tag,))

        self.count_label.config(text=f"Gefundene Spieler: {len(df_display)}")

    def sort_by_column(self, col):
        if self.sort_column == col:
            self.sort_reverse = not self.sort_reverse
        else:
            self.sort_column = col
            self.sort_reverse = False
        self.update_table()

    def on_search(self, event):
        self.apply_filters()

    def export_selected_players_as_json(self):
        selected_iids = self.tree.selection() # Erhält die iids (PERSON_ID) der ausgewählten Zeilen

        if not selected_iids:
            messagebox.showinfo("Export", "Bitte wählen Sie Spieler zum Exportieren aus.")
            return

        # Konvertiere iids zu einer Liste von Int64
        selected_player_ids = [int(iid) for iid in selected_iids]

        # Filtere den Haupt-DataFrame 'df' (oder 'self.filtered_df', beides funktioniert hier)
        # und wähle nur die 'DISPLAY_FIRST_LAST' Spalte aus
        players_names_to_export = self.filtered_df[
            self.filtered_df['PERSON_ID'].isin(selected_player_ids)
        ]['DISPLAY_FIRST_LAST'].tolist() # Konvertiere die Serie direkt in eine Liste

        # Dateidialog zum Speichern der Datei
        file_path = filedialog.asksaveasfilename(
            defaultextension=".json",
            filetypes=[("JSON files", "*.json"), ("All files", "*.*")],
            title="Spieler-Namen als JSON exportieren"
        )

        if file_path:
            try:
                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump(players_names_to_export, f, indent=4, ensure_ascii=False)
                messagebox.showinfo("Export", f"Spielernamen erfolgreich nach '{file_path}' exportiert.")
            except Exception as e:
                messagebox.showerror("Export Fehler", f"Fehler beim Exportieren: {e}")
        else:
            messagebox.showinfo("Export", "Export abgebrochen.")

# --- Start GUI ---
root = tk.Tk()
app = EfficiencyApp(root)
root.mainloop()



--- Initialer Lade-Check ---
Anzahl Spieler in df (nba_players_2023_2024): 572
Anzahl Spieler in salary_df (player_mit_salary): 573
Erste 3 Zeilen von salary_df:
   PERSON_ID FIRST_NAME LAST_NAME DISPLAY_FIRST_LAST DISPLAY_LAST_COMMA_FIRST  \
0    1630173   Precious   Achiuwa   Precious Achiuwa        Achiuwa, Precious   
1     203500     Steven     Adams       Steven Adams            Adams, Steven   
2    1628389        Bam   Adebayo        Bam Adebayo             Adebayo, Bam   

  DISPLAY_FI_LAST       PLAYER_SLUG            BIRTHDATE      SCHOOL  \
0      P. Achiuwa  precious-achiuwa  1999-09-19T00:00:00     Memphis   
1        S. Adams      steven-adams  1993-07-20T00:00:00  Pittsburgh   
2      B. Adebayo       bam-adebayo  1997-07-18T00:00:00    Kentucky   

       COUNTRY  ... TO_YEAR DLEAGUE_FLAG  NBA_FLAG  GAMES_PLAYED_FLAG  \
0      Nigeria  ...    2024            N         Y                  Y   
1  New Zealand  ...    2024            N         Y                  Y   
2   